# 激活统计（Activation Stats）学习笔记

基于 `10_activations.ipynb` 的学习总结。

## 1. 核心概念：为什么要关注激活值？

在深度神经网络中，每一层的输出（即**激活值**）是判断模型是否能有效训练的关键信号。两个经典的病态现象：

- **激活值爆炸（Exploding Activations）**：如果权重初始化过大，每一层的输出会指数级增长，最终变成 `nan`。
- **激活值消失（Vanishing Activations）**：如果权重初始化过小，信号逐层缩小直到趋近于零，深层网络完全无法学到东西。

理想状态是：**每一层的激活值均值接近 0、标准差接近 1**，这样信号能够稳定地在网络中传播。

## 2. PyTorch Hooks：非侵入式的诊断工具

### 2.1 什么是 Hook？

Hook 是 PyTorch 提供的回调机制，可以注册到任意 `nn.Module` 上，在前向传播（forward hook）或反向传播（backward hook）时自动执行。**不需要修改模型代码**就能收集中间层的信息。

### 2.2 演进过程

讲义展示了从简单到完善的三步演进：

**第一步：手动插入统计收集**

自定义 `SequentialModel`，在 `__call__` 中手动记录每层的 mean 和 std。缺点是需要修改模型本身。

```python
class SequentialModel(nn.Module):
    def __call__(self, x):
        for i, l in enumerate(self.layers):
            x = l(x)
            self.act_means[i].append(x.mean())
            self.act_stds[i].append(x.std())
        return x
```

**第二步：使用 PyTorch 原生 Hook**

通过 `register_forward_hook` 注册回调函数，hook 函数签名为 `(module, input, output)`：

```python
def append_stats(i, mod, inp, outp):
    act_means[i].append(outp.mean())
    act_stds[i].append(outp.std())

for i, m in enumerate(model):
    m.register_forward_hook(partial(append_stats, i))
```

**第三步：封装为 Hook/Hooks/HooksCallback 类**

- `Hook`：单个 hook 的封装，自动在删除时 remove（防内存泄漏）
- `Hooks`：hook 列表，支持 context manager（`with` 语句），自动清理
- `HooksCallback`：与 Learner 训练循环集成的 Callback，可以通过 `mod_filter` 选择只监控特定类型的层（如 `nn.Conv2d`）

## 3. 三种诊断可视化

最终封装为 `ActivationStats` 类，提供三种可视化方法：

### 3.1 `plot_stats()`：均值和标准差曲线

绘制每一层激活值的 **均值（mean）** 和 **标准差（std）** 随训练 batch 的变化曲线。

**如何判断好坏：**
- **好**：所有层的均值稳定在 0 附近，标准差稳定在 1 附近，各层曲线接近
- **坏**：均值逐渐偏移（趋向 0 或发散），标准差逐渐塌缩到 0（信号消失）或爆炸
- 基线 CNN 的问题：深层的均值和标准差都迅速降到接近 0，说明信号在传播中消亡

### 3.2 `color_dim()`：激活值直方图热力图

每一层一个子图，展示激活值的分布如何随训练变化：

- **横轴**：训练时间（batch 编号）
- **纵轴**：激活值的绝对值大小（0 在底部，10 在顶部，分 40 个 bin）
- **颜色亮度**：落在该区间的激活值数量（经 `log1p` 压缩）

**如何判断好坏：**
- **好**：颜色分布在一定范围内（如 0–3），并且在训练过程中保持稳定
- **坏**：颜色集中在最底部（接近 0），说明大量激活值为零，神经元已经死掉
- 越深的层问题越严重，颜色越暗越窄

### 3.3 `dead_chart()`：死亡神经元比例

绘制每层中**接近零的激活值所占比例**随训练的变化：

```python
def get_min(h):
    h1 = torch.stack(h.stats[2]).t().float()
    return h1[0] / h1.sum(0)  # 最低 bin 的占比
```

**如何判断好坏：**
- **好**：比例始终接近 0（接近零的激活很少）
- **坏**：比例趋向 1.0（几乎所有激活都是零），该层已经死掉了
- ReLU 的问题：将所有负值截断为 0，一旦神经元输出持续为负，梯度为零，永远无法恢复

## 4. 基线 CNN 的问题诊断

讲义中的基线模型使用了 5 层 strided Conv2d + ReLU，以 lr=0.6 的高学习率训练：

```python
def conv(ni, nf, ks=3, act=True):
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res
```

通过三种可视化可以观察到以下问题：

| 可视化 | 观察到的现象 | 说明 |
|--------|-------------|------|
| `plot_stats()` | 深层 mean 和 std 塌缩到接近 0 | 信号在深层消失 |
| `color_dim()` | 深层热力图颜色集中在底部 | 激活值大部分为零 |
| `dead_chart()` | 深层死亡比例趋向 1.0 | 几乎所有神经元都死了 |

**根本原因：**
1. PyTorch 默认的 Kaiming 初始化未必完全匹配当前网络配置
2. ReLU 将负值截断为 0，导致均值偏移（不再以 0 为中心），方差关系被破坏
3. 高学习率加速了这个恶性循环

## 5. 优化方案（后续讲义 11_initializing 中介绍）

针对上述问题，课程介绍了一系列递进的解决方案：

### 5.1 Kaiming/He 初始化

原理：对于 ReLU 激活函数，权重应按 `sqrt(2/n_in)` 缩放，以补偿 ReLU 截断一半输出的效果。

```python
def init_weights(m, leaky=0.):
    if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Conv3d)):
        init.kaiming_normal_(m.weight, a=leaky)
```

### 5.2 输入归一化

确保输入数据的均值为 0、标准差为 1，让第一层就能接收到合理缩放的输入。

```python
def _norm(b): return (b[0] - xmean) / xstd, b[1]
norm = BatchTransformCB(_norm)
```

### 5.3 GeneralReLU（Leaky + Subtract）

改良的激活函数，解决 ReLU 的两大问题：

```python
class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None): ...
    def forward(self, x):
        x = F.leaky_relu(x, self.leak) if self.leak else F.relu(x)
        if self.sub: x -= self.sub
        if self.maxv: x.clamp_max_(self.maxv)
        return x

act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)
```

- `leak=0.1`：Leaky ReLU，负值不完全截断，防止神经元死亡
- `sub=0.4`：减去常数，将激活值重新中心化到 0 附近

### 5.4 LSUV（Layer-wise Sequential Unit Variance）

数据驱动的初始化方法：在训练前，用一个真实 mini-batch 逐层调整权重，使每层的激活均值约等于0、标准差约等于1。

```python
def lsuv_init(model, m, m_in, xb):
    h = Hook(m, _lsuv_stats)
    with torch.no_grad():
        while abs(h.std - 1) > 1e-3 or abs(h.mean) > 1e-3:
            m_in.bias -= h.mean
            m_in.weight.data /= h.std
    h.remove()
```

### 5.5 Batch Normalization

最终的通用解决方案：在每个卷积层后加入 BatchNorm，**在训练过程中持续归一化激活值**（而不仅是初始化时）。

```python
def conv(ni, nf, ks=3, stride=2, act=nn.ReLU, norm=None, bias=None):
    layers = [nn.Conv2d(ni, nf, stride=stride, kernel_size=ks, padding=ks//2, bias=bias)]
    if norm: layers.append(norm(nf))
    if act: layers.append(act())
    return nn.Sequential(*layers)
```

使用 BatchNorm 后，模型在 FashionMNIST 上可以达到约 90% 的准确率。

### 优化方案总结

| 方案 | 类型 | 核心思路 |
|------|------|----------|
| Kaiming Init | 初始化 | 按理论公式缩放初始权重 |
| 输入归一化 | 数据预处理 | 确保第一层输入标准化 |
| GeneralReLU | 激活函数改良 | Leaky 防死亡 + Subtract 重中心化 |
| LSUV | 数据驱动初始化 | 用真实数据逐层校准 |
| BatchNorm | 训练中归一化 | 每个 batch 持续归一化，最稳健 |